# Creation of Vocabulary for STARR-MINER

This notebooks contains the code to create the vocabulary used for our STARR-MINER framework. To get more deatils about this framework and how it is used to populate the NOTE_NLP table in STARR-OMOP feel free to refer to our pre-print [here](https://arxiv.org/abs/2003.10534)

Below are the steps to create a new vocabulary
1. Filter the lexicons for:
    - Stopwords
    - Very long and short characters
    - Only numbers
    - Concept names with non alphanumeric characters in certain positions
    - Select a subset of vocabularies present in the OHDSI vocabulary. 
2. The primary key is the lowercased version of MRCONSO.STR 
3. We map from CUI's to concept_id a SQL code developed in the OHDSI package [Ananke](https://github.com/jmbanda/Ananke). 

The paper that describes some of the steps taken here to filter UMLS to produce a vocabulary to be used with clinical NLP pipelines can be found [here](https://pubmed.ncbi.nlm.nih.gov/22493050/)


In [1]:
from google.cloud import bigquery
import pandas as pd
import os
import re
import spacy
import glob

In [2]:
json_path = "/Users/jdposada/.config/gcloud/application_default_credentials.json"
service_account_path = "/Users/jdposada/Documents/secret"
cdm_project_id = 'som-rit-phi-starr-prod'
cdm_dataset_id = 'starr_omop_cdm5_deid_latest'
omop_vocab_dataset_id = 'vocabulary_20200717'
work_project_id = 'som-rit-phi-starr-miner-dev'
work_dataset_id = 'jdposada_explore'
umls_dataset_id = 'umls_2019AA'
starr_miner_vocab_name = '20200805_starr_miner_dictionary'
csv_files_folder = "../resources"
sql_files_folder = "../resources"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_path
os.environ['GCLOUD_PROJECT'] = work_project_id

In [3]:
client = bigquery.Client(project=work_project_id);

## Filters based on char length, words, only numbers

**Read the parenthesis list**. This is names that contains string like uta (disorder)

In [4]:
parenthesis_list = open(f"{csv_files_folder}/parenthesis_list.csv", 'r').readlines()
parenthesis_list = [re.sub(r'\n', '', x) for x in parenthesis_list]
parenthesis_list = '|'.join(parenthesis_list)

**Read and update stopwords with spacy**

In [5]:
stopwords_list = open(f"{csv_files_folder}/stopwords.csv", encoding='utf-8', mode='r+').readlines()
stopwords_list = [re.sub(r'\n', '', x.lower()) for x in stopwords_list]
from spacy.lang.en.stop_words import STOP_WORDS
stopwords_list = list(set(stopwords_list).union(set(STOP_WORDS)))
stopwords_list.sort()
with open('stopwords.csv',encoding='utf-8', mode='w+') as f:
    [f.write(x + '\n') for x in stopwords_list]

Upoload to BQ

In [6]:
stopwords_list_df = pd.DataFrame({'stopword': stopwords_list})
client.load_table_from_dataframe(dataframe = stopwords_list_df, 
                                 destination = f'{work_project_id}.{work_dataset_id}.stopwords');

**Read the list of vocabularies**

In [7]:
vocabularies_df = pd.read_csv(f"{csv_files_folder}/sab_list.csv")
sab_list = vocabularies_df['SAB'].values.tolist()
sab_list = "('" + "','".join(sab_list) + "')"

In [8]:
regex_list = [x.split('\n')[0] for x in open(f"{csv_files_folder}/regex_list.txt", 'r').readlines()]

**Create the SQL string and execute**

In [9]:
%%time
sql = """
CREATE OR REPLACE TABLE
  `{work_project_id}.{work_dataset_id}.mrconso_filtered` AS
SELECT
  *
FROM (
  SELECT
    mrconso.sui,
    mrconso.cui,
    mrconso.code,
    mrconso.lat,
    mrsty.tui,
    mrsty.sty,
    mrconso.sab,
    mrconso.str,
    LOWER(str) AS lowerstr,
    LENGTH(str) AS charlen,
    ARRAY_LENGTH(REGEXP_EXTRACT_ALL(str, '\\\S{{1,}}')) AS word_count
  FROM
    `{work_project_id}.{umls_dataset_id}.MRCONSO` mrconso
  JOIN
  `{work_project_id}.{umls_dataset_id}.MRSTY` mrsty
   ON
    mrsty.cui = mrconso.cui
  WHERE
    lat = 'ENG' -- 10,406,797
    AND mrconso.SAB in {sab_list}
    AND NOT REGEXP_CONTAINS(str, r'{rgx[0]}') 
    AND NOT REGEXP_CONTAINS(str, r'{rgx[1]}') 
    AND NOT REGEXP_CONTAINS(str, r'{rgx[2]}') 
    AND NOT REGEXP_CONTAINS(LOWER(str), r'\({parenthesis_list}\)') 
    AND LOWER(str) NOT IN 
    (SELECT 
      LOWER(stopword) 
     FROM
      `{work_project_id}.{work_dataset_id}.stopwords`)
    )
WHERE
  charlen > 2 
  AND charlen < 56 
  AND word_count < 7 
""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'umls_dataset_id': umls_dataset_id,
                'parenthesis_list': parenthesis_list,
                'sab_list': sab_list,
                'rgx': regex_list})

client.query(sql).result();

CPU times: user 31.8 ms, sys: 5.44 ms, total: 37.3 ms
Wall time: 1min 26s


## Use the concept_id to CUI mappings from Ananke

Copy the vocab to the work project

In [10]:
dataset_ref = client.dataset(dataset_id=omop_vocab_dataset_id, 
                             project=cdm_project_id)
table_list = client.list_tables(dataset_ref)
tables = [x for x in table_list]
table_id_list = [x.table_id for x in tables]

# create the destination dataset
client.create_dataset(f"{work_project_id}.{omop_vocab_dataset_id}", exists_ok=True)
dest_dataset_ref = client.dataset(dataset_id=omop_vocab_dataset_id, 
                                  project=work_project_id)
for table_id in table_id_list:
    source_table_ref = dataset_ref.table(table_id)
    dest_table_ref = dest_dataset_ref.table(table_id)
    job_config = bigquery.CopyJobConfig()
    job_config.write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
    job = client.copy_table(
        source_table_ref,
        dest_table_ref,
        # Location must match that of the source and destination tables.
        location='US',
        job_config=job_config)
    job.result()

In [11]:
sql_map = open(f"{sql_files_folder}/cui2ohdsi_concept_idv1.1_mod.sql", 'r').read()

In [12]:
sql_map = re.sub(pattern = r'OHDSIVocab.OHDSI_to_CUI_TEMP', 
                 repl= f"`{work_project_id}.{omop_vocab_dataset_id}.ohdsi_to_cui_temp`", 
                 string = sql_map)

sql_map = re.sub(pattern = r'OHDSIVocab.OHDSI_to_CUI', 
                 repl= f"`{work_project_id}.{omop_vocab_dataset_id}.ohdsi_to_cui`", 
                 string = sql_map)

sql_map = re.sub(pattern = r'OHDSIVocab.concept', 
                 repl= f"`{work_project_id}.{omop_vocab_dataset_id}.concept`", 
                 string = sql_map)

sql_map = re.sub(pattern = r'umls.MRCONSO', 
                 repl= f"`{work_project_id}.{work_dataset_id}.mrconso_filtered`", 
                 string = sql_map)

sql_map = re.sub(pattern = r'UNION', 
                 repl= "UNION ALL", 
                 string = sql_map)

sql_map = re.sub(pattern = r'GROUP BY AA.CUI', 
                 repl= "GROUP BY AA.CUI, AA.concept_id, AA.vocabulary_id", 
                 string = sql_map)

sql_map = re.sub(pattern = r'GROUP BY A.CUI', 
                 repl= "GROUP BY A.CUI, B.concept_id", 
                 string = sql_map)

**Finally create the mappings. This creates a table called ohdsi_to_cui**

In [13]:
%%time
client.query(sql_map).result();

CPU times: user 39 ms, sys: 7.87 ms, total: 46.8 ms
Wall time: 3min 13s


**Now let's do the join and get each string a concept_id**

In [14]:
%%time
sql = """
CREATE OR REPLACE TABLE
 `{work_project_id}.{work_dataset_id}.mrconso_filtered_concept_id` AS
SELECT
 mrconso.*,
 map_.concept_id
FROM
  `{work_project_id}.{work_dataset_id}.mrconso_filtered` mrconso
JOIN
  `{work_project_id}.{omop_vocab_dataset_id}.ohdsi_to_cui` map_
ON
  mrconso.cui = map_.cui
""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'omop_vocab_dataset_id': omop_vocab_dataset_id})

client.query(sql).result();

CPU times: user 17.5 ms, sys: 3.41 ms, total: 20.9 ms
Wall time: 29.3 s


## Let's create the final dictionary

In [15]:
%%time
sql = """
CREATE OR REPLACE TABLE
 `{work_project_id}.{work_dataset_id}.temp_starr_miner_vocab` AS
SELECT
 ROW_NUMBER() OVER() AS concept_index,
 ARRAY_AGG(DISTINCT concept_id) AS concept_id_array,
 lowerstr
FROM
  `{work_project_id}.{work_dataset_id}.mrconso_sui_concept_id`
GROUP BY
 lowerstr;

CREATE OR REPLACE TABLE
 `{work_project_id}.{work_dataset_id}.{starr_miner_vocab_name}` AS
SELECT
  concept_index,
  concept_id,
  lowerstr
FROM
  `{work_project_id}.{work_dataset_id}.temp_starr_miner_vocab` as t
CROSS JOIN
  UNNEST(t.concept_id_array) AS concept_id;
  
CREATE OR REPLACE TABLE
 `{work_project_id}.{work_dataset_id}.{starr_miner_vocab_name}` AS
SELECT
  concept_index,
  concept_id,
  COALESCE(cr.concept_id_2, 0) AS standard_concept_id,
  lowerstr
FROM
  `{work_project_id}.{work_dataset_id}.{starr_miner_vocab_name}` as t
LEFT JOIN
 `{work_project_id}.{omop_vocab_dataset_id}.concept_relationship` cr
ON
 cr.concept_id_1 = t.concept_id
 AND cr.relationship_id = 'Maps to';

DROP TABLE IF EXISTS `{work_project_id}.{work_dataset_id}.temp_starr_miner_vocab`;

""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'starr_miner_vocab_name': starr_miner_vocab_name,
                'omop_vocab_dataset_id': omop_vocab_dataset_id})

client.query(sql).result();

CPU times: user 35.6 ms, sys: 6.62 ms, total: 42.3 ms
Wall time: 3min 28s


Let's remove again stopwords again that could have been brought in after all the prior joins

In [16]:
sql = """
CREATE OR REPLACE TABLE
 `{work_project_id}.{work_dataset_id}.{starr_miner_vocab_name}` AS
SELECT
 vocab.*
FROM
 `{work_project_id}.{work_dataset_id}.{starr_miner_vocab_name}` vocab
LEFT JOIN
 `{work_project_id}.{work_dataset_id}.stopwords` stopwords
ON
 stopwords.stopword = vocab.lowerstr
WHERE 
 stopwords.stopword is NULL
""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'starr_miner_vocab_name': starr_miner_vocab_name})

client.query(sql).result();

Let's double check none of the stopwords are actually in the vocab

In [17]:
sql = """
SELECT
 COUNT(1) AS counts
FROM
 `{work_project_id}.{work_dataset_id}.{starr_miner_vocab_name}` vocab
JOIN
 `{work_project_id}.{work_dataset_id}.stopwords` stopwords
ON
 stopwords.stopword = vocab.lowerstr
""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'starr_miner_vocab_name': starr_miner_vocab_name})

df = client.query(sql).to_dataframe();
df

,counts
0,0


**Let's check how many rows do we get and how the final dictionary looks like**

In [18]:
sql = """
SELECT
 COUNT(DISTINCT lowerstr) n_unique_strings,
 COUNT(DISTINCT concept_id) n_unique_concept_id,
 COUNT(DISTINCT standard_concept_id) n_unique_standard_concept_id
FROM
 `{work_project_id}.{work_dataset_id}.{starr_miner_vocab_name}`
""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'starr_miner_vocab_name': starr_miner_vocab_name,
                'omop_vocab_dataset_id': omop_vocab_dataset_id})

df = client.query(sql).to_dataframe()

In [19]:
df

,n_unique_strings,n_unique_concept_id,n_unique_standard_concept_id
0,989293,749337,470656


Let's check how the vocab looks like

In [20]:
sql = """
SELECT * 
FROM
 `{work_project_id}.{work_dataset_id}.{starr_miner_vocab_name}`
LIMIT 5
""".format_map({'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'starr_miner_vocab_name': starr_miner_vocab_name,
                'omop_vocab_dataset_id': omop_vocab_dataset_id})

df = client.query(sql).to_dataframe()

In [21]:
df

,concept_index,concept_id,standard_concept_id,lowerstr
0,623722,443343,443343,female genital diseases
1,655579,4105418,4105418,ability to run uphill
2,37553,4293949,40236190,gg 100/pseudoephedrin 30mg/5ml alc-f syr
3,108689,4019486,4019486,fibromatous tumour
4,747705,4070191,4070191,eimeria rajesthani
